# Self-Explaining Neural Networks: A review with extensions
---

<img src='senn/notebooks/img/SENN.png' style='width: 500px;'/>

## Table of Contents

1. [Environment Setup](#setup)
2. [Reproducing MNIST Results](#reproduce_mnist)
3. [Reproducing COMPAS Results](#reproduce_compas)
4. [Study of Robustness](#robustness_study)
5. [Self-Explaining Neural Networks with Disentanglement](#DiSENN)
___

<a name="setup"></a>
# 1. Environment Setup

## Import libraries

In [1]:
import json
import torch
import numpy as np
import torch.nn as nn
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from pathlib import Path
from types import SimpleNamespace
from importlib import import_module

%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [ ]:
from senn.models.losses import *
from senn.models.parameterizers import *
from senn.models.conceptizers import *
from senn.models.aggregators import *
from senn.models.senn import SENN, DiSENN

In [ ]:
from senn.datasets.dataloaders import get_dataloader
from senn.utils.plot_utils import show_explainations, show_prototypes, plot_lambda_accuracy, get_comparison_plot

## Utility functions

In [ ]:
def get_config(filename):
    config_path = Path('configs')
    config_file = config_path / filename
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    return SimpleNamespace(**config)

In [ ]:
def load_checkpoint(config):
    model_file = Path('results') / config.exp_name / "checkpoints" / "best_model.pt" 
    return torch.load(model_file, config.device)

In [ ]:
def accuracy(model, dataloader, config):
    accuracies = []
    model.eval()
    with torch.no_grad():
        for i, (x, labels) in enumerate(dataloader):
            x = x.float().to(config.device)
            labels = labels.long().to(config.device)
            y_pred, (concepts, relevances), _ = model(x)
            accuracies.append((y_pred.argmax(axis=1) == labels).float().mean().item())
    accuracy = np.array(accuracies).mean()
    print(f"Test Mean Accuracy: {accuracy * 100: .3f} %")

## Configuration

In [ ]:
plt.style.use('seaborn-notebook')

<a name="reproduce_mnist"></a>
# 2. Reproducing MNIST Results

## Load MNIST Data and Config

In [ ]:
mnist_config = get_config("mnist_lambda1e-4_seed29.json")
mnist_config.device = "cpu"

In [ ]:
_, _, mnist_test_dl = get_dataloader(mnist_config)

## Load Trained Model

In [ ]:
conceptizer = ConvConceptizer(**mnist_config.__dict__)
parameterizer = ConvParameterizer(**mnist_config.__dict__)
aggregator = SumAggregator(**mnist_config.__dict__)

mnist_SENN = SENN(conceptizer, parameterizer, aggregator)

In [ ]:
mnist_checkpoint = load_checkpoint(mnist_config)
mnist_SENN.load_state_dict(mnist_checkpoint['model_state'])

## Test Accuracy

First we run and evaluate the prediction performance of our model to check whether the results reported by the authors are reproducible or not. We are using the same hyperparameters and architecture that were reported by the authors.

In [ ]:
accuracy(mnist_SENN, mnist_test_dl, mnist_config)

The obtained test accuracy closely matches the reported range of the authors which was 99.1% to 98.7% for different lambda values. We used $\lambda = 1e-4$  for the tested model. For further hyperparameter details you can consult the config files used or the table in the report. 

## Test Explanations

The following function randomly samples from a test batch and produces explanations for their classification. With this experiment we want to qualitatively examine the interpretibility of the generated explanations. The model used in this experiment is the same as above.

In [ ]:
show_explainations(mnist_SENN, mnist_test_dl, 'mnist')

The relevance scores of the explanations shown in the original paper are exclusively values close to either positive or negative one. The relevance scores in our explanations are more diverse.

The following function finds the top nine prototypical test examples that activate a certain concept the most. This visualization method was also used by the authors.

In [ ]:
show_prototypes(mnist_SENN, mnist_test_dl, 'activation')

In the original paper the prototypes of one concept are mostly of the same digit class. While this tendency can also be observed in some of our concept representations in other cases the concept is not prototypical for a certain digit class.

In general we do not consider the vast majority of the explanations to be plausible and human interpretable. Therefore, this experiment is not reproducible. We believe that the authors picked only some of the few plausible examples by hand rather than showing randomly sampled results. We consider the poor interpretebility of the explanations a severe limitation of the framework. We argue that the lack of interpretability is at least partially caused by the way of representing the concepts. Selecting prototypes based only on the highest activation for each concept seems for us to be a crude method that fails to capture the real semantic meaning of the concept.

## Extending Concept Representation Techniques

The authors proposed two alternative methods for representing concepts as future work. As we argue in the previous experiment that the concept visualization has potential for improovement we tryed both of these methods.

We start with another prototype visualization method which we call '$\textit{highest contrast}$'. The difference to the standard method (highest activation) is that the prototypes are not selected by only optimizing for highest activation for a certain concept but by also considering that all remaining concepts should be activated as little as possible at the same time.

The following function finds the prototypes with the highest contrast method using the same model as above for highest activation

In [ ]:
show_prototypes(mnist_SENN, mnist_test_dl, 'contrast')

We find that the prototypes are not more human interpretable than with the highest activation method. The efect of concepts not being represented by a single digit class is even stronger for the highest contrast method.
Note that the two visualization methods are inconsistent in some cases because the interpretation of the same underlying concept changes. A good example of this is Concept 4. In the highest activation method it represented the digit class three but with the highest contrast method it is represented by many different digit classes that could be interpreted as sharing the property of roundness. 
Finding an interpretation of the concept representation is left to the user and is therefore highly subjective. 

The next function uses the second approach proposed by the authors. This time the concepts are not represented by prototypes but by the weights of the concolutional concept encoders. We use the last layer of the encoder because deeper layers represent more high-level features.

In [ ]:
show_prototypes(mnist_SENN, mnist_test_dl, 'filter')

The filters do not exhibit any human interpretable features. Considering the limited depth of the concept encoder, relatively small filter size and the simplicity of the data this is not surprising. However, we think a more suitable approach would be to visualize the filter activations of transposed convolutions applied on the concept specific filters respectively.

## Tradeoff: Accuracy vs. Robustness 

In this experiment we want to analyze the negative impact of the robustness loss rgularization on the accuracy. We therefore incrementally increase the amount of regularization and training the same model for each of the $\lambda$-values. Because on MNIST the results were stable we only use one seed to report the results.

Below you can see the config files of the five models that are used to generate the plot.

In [ ]:
config_list = ['mnist_lambda1e-4_seed29.json', 'mnist_lambda1e-3_seed29.json', 'mnist_lambda1e-2_seed29.json', 
               'mnist_lambda1e-1_seed29.json', 'mnist_lambda1e-0_seed29.json']

The valid flag here is only used in this notebook to speed up calculations. The five models are not evaluated on the test set but instead the validation accuracies recorded during training are loaded. In the report we used the test accuracies which are almost the same as we did not do any hyperparameter search on the validation set.

In [ ]:
# NOTE: This will read the saved validation accuracies to speed up the process
# in the report we evaluate on the test set
_ = plot_lambda_accuracy(config_list, num_seeds=1, valid=True)

There is only a very small downward trend in accuracy up to the extreme value of $\lambda = 1$ where the accuracy drops a lot. The general trend is consistent with the authors result but they do not report the huge drop for $\lambda=1$.

<a name="reproduce_compas"></a>
# 3. Reproducing COMPAS Results

## Load Test Data and Configuration

In [ ]:
compas_config = get_config("compas_lambda1e-4_seed555.json")
compas_config.device = "cpu"

In [ ]:
_, _, compas_test_dl = get_dataloader(compas_config)

## Load Trained Model

In [ ]:
conceptizer = IdentityConceptizer(**compas_config.__dict__)
parameterizer = LinearParameterizer(**compas_config.__dict__)
aggregator = SumAggregator(**compas_config.__dict__)

compas_SENN = SENN(conceptizer, parameterizer, aggregator)

In [ ]:
compas_checkpoint = load_checkpoint(compas_config)

compas_SENN.load_state_dict(compas_checkpoint['model_state'])

## Test Accuracy

Same as for MNIST we run and evaluate the prediction performance of our COMPAS model to check whether the results reported by the authors are reproducible or not. We are using the same hyperparameters and architecture that were reported by the authors for this dataset again.

In [ ]:
accuracy(compas_SENN, compas_test_dl, compas_config)

The obtained test accuracy closely matches the reported result of the authors (82%). We used $\lambda = 1e-4$  for the tested model. For further hyperparameter details you can consult the config files used or the table in the report. 

## Test Explanations

For completeness sake we also visualize two examples of explanations for the COMPAS dataset. As here no concepts are learned ( instead the raw inputs are used) the analysis of these explanations is not that interesting and therefore only shown in the reprt's appendix.

In [ ]:
# show_explainations(compas_SENN, compas_test_dl, 'compas', concept_names=compas_config.concept_names)
show_explainations(compas_SENN, compas_test_dl, 'compas', concept_names=[f"C{i}" for i in range(11)])

## Tradeoff: Accuracy vs. Robustness

Analogously, to the regularization study performed on MNIST (see above) we do the same for the COMPAS dataset. The difference is that we train the model on three different seeds per lambda value. The used config files are listed below.

In [ ]:
config_list = [["compas_lambda1e-4_seed111.json","compas_lambda1e-4_seed333.json","compas_lambda1e-4_seed555.json"],
               ["compas_lambda1e-3_seed111.json","compas_lambda1e-3_seed333.json","compas_lambda1e-3_seed555.json"],
               ["compas_lambda1e-2_seed111.json","compas_lambda1e-2_seed333.json","compas_lambda1e-2_seed555.json"],
               ["compas_lambda1e-1_seed111.json","compas_lambda1e-1_seed333.json","compas_lambda1e-1_seed555.json"],
               ["compas_lambda1e-0_seed111.json","compas_lambda1e-0_seed333.json","compas_lambda1e-0_seed555.json"]]

Again we use the valid flag in this notebook to speed up computations (see explanation  in the respective MNIST experiment)

In [ ]:
# NOTE: This will read the saved validation accuracies to speed up the process
# in the repoert we evaluate on the test set
_ = plot_lambda_accuracy(config_list, num_seeds=3, valid=True)

We observe the same downward trend in accuracy for increasing regularization that was reported by the authors. However, the magnitude of the decrease is a lot higher in our experiments. The authors only report a drop of about 4% points while in our experiment the difference in accuracy between the two extreme values is approximately 15% points. 

<a name="robustness_study"></a>
# 4. Study of Robustness

Inspired by the robustness study the authors conducted by perturbating an input image with Guassian noise, we decided to take a different approach. Instead of using the input image with different levels of noise, we try to find semantically similar images in the dataset and plot their concepts and relevance scores. To cluster the images we train an autoencoder, whose architecture can be found in the appendix of the report, and perform k-nearest-neighbour search of the latent space (Euclidean distance) to get the images from the dataset that look most similar to the query image. As a result, we get k + 1 images for which we compare the concept activation and relevance scores.

## Load pre-trained MNIST autoencoder

In [ ]:
from senn.utils.MNIST_autoencoder import AETrainer, get_most_similar

ae_trainer = AETrainer(mnist_test_dl, batch_size=200)
ae_trainer.load_model("senn/utils/MNIST_autoencoder_pretrained.pt")

**Get the latent vector for all the inputs from the test set**

In [ ]:
mnist_latents = ae_trainer.get_latent_reps(mnist_test_dl)

## Test robustness for visually similar images

In [ ]:
# Get the indices of the closest vectors
nearest_num = 5
query_index = 0

distances, indices = get_most_similar(latents=mnist_latents,
                                      query=mnist_latents[query_index],
                                      number=nearest_num)

In [ ]:
# prepare input images from the testloader 
test_images = torch.stack([mnist_test_dl.dataset[index][0] for index in indices[0]])

fig = get_comparison_plot(test_images, mnist_SENN)
fig.tight_layout()

The experiment shows that the generated explanations are robust because the relevances as well as the concept activations are very similar for these perceptually similar test examples.

<a name="DiSENN"></a>
# 5. Self-Explaining Neural Networks with Disentanglement

DiSENN incorporates a constrained variational inference framework on a 
SENN Concept Encoder to learn disentangled representations of the 
basis concepts as in [2]. The basis concepts are then independently
sensitive to single generative factors leading to better interpretability 
and lesser overlap with other basis concepts. Such a strong constraint 
better fulfills the "diversity" desiderata for basis concepts
in a Self-Explaining Neural Network.


References  
[1] Alvarez Melis, et al.
"Towards Robust Interpretability with Self-Explaining Neural Networks" NIPS 2018  
[2] Irina Higgins, et al. 
”β-VAE: Learning basic visual concepts with a constrained variational framework.” ICLR 2017. 

## Load Data and Configuration

In [ ]:
disenn_config = get_config("MNIST_DiSENN.json")

In [ ]:
_, _, mnist_test_dl = get_dataloader(disenn_config)

## Load Trained Model

In [ ]:
conceptizer = VaeConceptizer(**disenn_config.__dict__)
parameterizer = ConvParameterizer(**disenn_config.__dict__)
aggregator = SumAggregator(**disenn_config.__dict__)

disenn = DiSENN(conceptizer, parameterizer, aggregator)

In [ ]:
disenn_checkpoint = load_checkpoint(disenn_config)
disenn.load_state_dict(disenn_checkpoint['model_state'])

## Test Accuracy

In [ ]:
accuracy(disenn, mnist_test_dl, disenn_config)

## Test Explanations

**Generate Prototypes from Disentangled Concepts**

In [ ]:
xb, yb = next(iter(mnist_test_dl))

**Find all images of digit 3**

In [ ]:
(yb==3).nonzero()

In [ ]:
x = xb[18].cpu()
fname = "/digit3a.png"
disenn.explain(x, contrast_class=8, traversal_range=0.45,
              gridsize=(1,6), col_span=3, figure_size=(18,3), show=True,
              save_as="results/"+disenn_config.exp_name+fname, use_cdf=True)

In [ ]:
x = xb[30].cpu()
fname = "/digit3b.png"
disenn.explain(x, contrast_class=8, traversal_range=0.45,
              gridsize=(1,6), col_span=3, figure_size=(18,3), show=True,
              save_as="results/"+disenn_config.exp_name+fname, use_cdf=True)

**A random digit**

In [ ]:
x = xb[0].cpu()
fname = "/digit.png"
disenn.explain(x, contrast_class=0, traversal_range=0.45,
              gridsize=(1,6), col_span=3, figure_size=(18,3), show=True,
              save_as="results/"+disenn_config.exp_name+fname, use_cdf=True)